In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import string
from datetime import datetime, timedelta
from selenium.webdriver import ActionChains
import pyodbc
import win32com.client as win32
import undetected_chromedriver as uc

servico = Service(ChromeDriverManager().install())
options= webdriver.ChromeOptions()
# options.add_argument(r'--user-data-dir=C:\Users\ARTHUR\AppData\Local\Google\Chrome\User Data\Profile 4')
navegador = uc.Chrome(service=servico,options=options)
navegador.maximize_window()
navegador.execute_script("window.open('');")
navegador.get('https://www.livelo.com.br/SAML/post')

WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=131.0.6778.140)
Stacktrace:
	GetHandleVerifier [0x005134A3+25059]
	(No symbol) [0x0049CEA4]
	(No symbol) [0x0037BD39]
	(No symbol) [0x0036D440]
	(No symbol) [0x0036B91B]
	(No symbol) [0x0036BF8D]
	(No symbol) [0x00378647]
	(No symbol) [0x00387905]
	(No symbol) [0x0038C766]
	(No symbol) [0x0036C595]
	(No symbol) [0x00387537]
	(No symbol) [0x003FB854]
	(No symbol) [0x003E1C96]
	(No symbol) [0x003B3FAC]
	(No symbol) [0x003B4F3D]
	GetHandleVerifier [0x00805613+3113811]
	GetHandleVerifier [0x0081A2DA+3199002]
	GetHandleVerifier [0x00812AB2+3168242]
	GetHandleVerifier [0x005B3310+680016]
	(No symbol) [0x004A57ED]
	(No symbol) [0x004A2A98]
	(No symbol) [0x004A2C35]
	(No symbol) [0x00495890]
	BaseThreadInitThunk [0x75987BA9+25]
	RtlInitializeExceptionChain [0x779BC0CB+107]
	RtlClearBits [0x779BC04F+191]


In [35]:
def esperar_tempo(tipo,code):
    if tipo == 'id':
        contador = navegador.find_elements(By.ID,code)
        while len(contador)<1:
            time.sleep(0.2)
            contador = navegador.find_elements(By.ID,code)
            try:
                erro = navegador.find_element(By.TAG_NAME,'h1').text()
                if erro == '502 Bad Gateway':
                    navegador.refresh()
            except:
                None
    elif tipo=='class':
        contador = navegador.find_elements(By.CLASS_NAME,code)
        while len(contador)<1:
            time.sleep(0.2)
            contador = navegador.find_elements(By.CLASS_NAME,code)
    elif tipo == 'xpath':
        contador = navegador.find_elements(By.XPATH,code)
        while len(contador)<1:
            time.sleep(0.2)
            contador = navegador.find_elements(By.XPATH,code)
    else:
        print(f'Esse tipo é invalido ou foi escrito de forma errada')

def lista_precos(lista):
    precos_ajustados = []
    for item in lista:
        preco = item.text
        # print(preco)
        if '+' in preco:
            posicao = preco.find('+')
            adicional = preco[posicao:]
            preco = preco.replace(adicional,'')
        preco = preco.replace('Total pago','').replace('\n','').replace('R$ ','').replace('.','').replace(',','.')
        precos_ajustados.append(preco)
    return precos_ajustados

def passar_pagina_extratos(o):
    navegador.find_element(By.ID,'pagination')
    if o>=2:
        elemento = navegador.find_element(By.XPATH,'//*[@id="pagination"]/a[7]')
        navegador.execute_script("arguments[0].click();", elemento)
    else:
        # navegador.find_element(By.XPATH,'//*[@id="pagination"]/a[6]').click()
        elemento = navegador.find_element(By.XPATH, '//*[@id="pagination"]/a[6]')
        navegador.execute_script("arguments[0].click();", elemento)

def passar_pagina_pedidos(u):
    # navegador.find_element(By.ID,'pagination')
    if u>=2:
        elemento = navegador.find_element(By.XPATH,'//*[@id="wi31300094-pagination-orders-grid-v2-id"]/div/a[7]')
        navegador.execute_script("arguments[0].click();", elemento)
    else:
        elemento = navegador.find_element(By.XPATH, '//*[@id="wi31300094-pagination-orders-grid-v2-id"]/div/a[6]')
        navegador.execute_script("arguments[0].click();", elemento)

def verificador_somas(linha,tabela):
    if linha in tabela.index:
        op_valor = tabela.loc[linha,'Valor']
        op_qtd = tabela.loc[linha,'Quantidade']
    else:
        op_valor, op_qtd = [0,0]
    return op_valor,op_qtd

In [36]:
navegador.get('https://www.livelo.com.br/SAML/post')
esperar_tempo('id','div-controlLogin')

dados_conexao= ('Driver={MySQL ODBC 9.1 ANSI Driver};Server=srv1124.hstgr.io;Database=u785316102_database_heix;UID=u785316102_Helles;PWD=Pepin@s1')
conn =pyodbc.connect(dados_conexao)
cursor = conn.cursor()

cursor.execute("SELECT * FROM Contas WHERE Empresa='Livelo'")
dados = cursor.fetchall()
colunas = [column[0] for column in cursor.description]
livelo_df=pd.DataFrame([tuple(row) for row in dados], columns=colunas)
cursor.execute("SELECT Id, Nome FROM Pessoas")
pessoas = cursor.fetchall()
colunas = [column[0] for column in cursor.description]
pessoas_df=pd.DataFrame([tuple(row) for row in pessoas], columns=colunas)
cursor.close()
conn.close()

ontem = (datetime.now() - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
anteontem = ontem - timedelta(days=1)
texto=''

for l in range(livelo_df.shape[0]):
    contas={}
    id_conta= int(livelo_df.iloc[l,0])
    id_pessoa = livelo_df.iloc[l,1]
    username = livelo_df.iloc[l,2]
    senha = livelo_df.iloc[l,3]
    print(l)
    
    if l!=0:
        esperar_tempo('id','div-controlLogin')
    navegador.find_element(By.ID,'div-controlLogin').click()
    esperar_tempo('id','username')
    
    navegador.find_element(By.ID,'username').send_keys(username)
    time.sleep(0.75)
    navegador.find_element(By.ID, 'password').send_keys(senha)
    navegador.find_element(By.ID,'btn-submit').click()
    time.sleep(1.5)
    try: 
        alo = navegador.find_element(By.XPATH,'//*[@id="token-selection-form"]/div/div[2]/div/div').text
        if alo =='Onde você prefere receber o seu código?':
            navegador.get('https://www.livelo.com.br/SAML/post')
            esperar_tempo('id','div-controlLogin')
            continue
    except:
        None
    
    while True:
        time.sleep(0.2)
        first = navegador.find_elements(By.ID,'span-firstNameText')
        nick = navegador.find_elements(By.ID,'span-nicknameText')
        if (len(first)+len(nick))>0:
            break


    navegador.get('https://www.livelo.com.br/extrato')
    navegador.switch_to.window(navegador.window_handles[1])
    navegador.get('https://www.livelo.com.br/orderHistory')
    x = input('BOTA 1')
    while True:
        if x != '1':
            x = input('BOTA 1')
        else:
            break
    esperar_tempo('class','order-card__total')

    precos = navegador.find_elements(By.CLASS_NAME,'order-card__total')
    precos_ajustados = lista_precos(precos)
    navegador.switch_to.window(navegador.window_handles[0])
    esperar_tempo('class','transactions__item')
    #talvez tenha que mudar a estrutura do codigo, aonde difere por pontos+dinheiro do resto! (nem precisou mudar)

    movimentos = {}
    ultima_atualização = datetime.strptime(str(livelo_df.iloc[l,6]),'%Y-%m-%d')
    operacao_sem_valor = 'Membership RewardsFidelidade Bradesco NATURA'   
    transferiveis = 'Smiles LATAM Pass Azul'
    atualizacao= False
    pedido = 0
    i=1
    o=0
    u=0
    valor_clube = 0
    
    while True:
        #pegando as informações de cada coluna
        data = navegador.find_element(By.XPATH,f'//*[@id="transactions"]/table/tbody/tr[{i}]/td[1]').text
        data_operacional = datetime.strptime(data, '%d/%m/%Y')
        if data_operacional<=ultima_atualização:
            break
        elif data_operacional>= ontem:
            i+=1
            continue
        operacao = navegador.find_element(By.XPATH,f'//*[@id="transactions"]/table/tbody/tr[{i}]/td[2]').text
        parceiro = navegador.find_element(By.XPATH,f'//*[@id="transactions"]/table/tbody/tr[{i}]/td[3]').text
        if parceiro == 'Pontos + Dinheiro':
            parceiro = parceiro_saida
        elif parceiro in transferiveis:
            parceiro_saida = parceiro
        pontos = navegador.find_element(By.XPATH,f'//*[@id="transactions"]/table/tbody/tr[{i}]/td[4]').text
        if 'Turbinado' in pontos:
            cortar = pontos.find('Turbinado')
            cortavel = pontos[cortar:]
            pontos = pontos.replace(cortavel,'')
        for a in pontos:
            if a in string.ascii_letters or a in string.punctuation or a in ' ,\n':
                pontos= pontos.replace(a,'')
        pontos = int(pontos)
        
        #relacionamento de cada linha com um valor  
        if operacao == 'Resgate' or parceiro =='COOPERA':
            valor = 'Calcular'
        else:
            if parceiro in operacao_sem_valor or operacao=='Bônus':
                valor=0
            elif id_conta == 88 and parceiro == 'Clube Livelo':
                valor = livelo_df.iloc[l,10]
            else:
                while 'pontos' in precos_ajustados[pedido]:
                    pedido+=1
                    if pedido>=10:
                        #troca de pagina, não mudar
                        pedido=0
                        navegador.switch_to.window(navegador.window_handles[1])
                        passar_pagina_pedidos(u)
                        u+=1
                        esperar_tempo('class','order-card__total')
                        time.sleep(2)
                        precos = navegador.find_elements(By.CLASS_NAME,'order-card__total')
                        precos_ajustados=lista_precos(precos)
                        navegador.switch_to.window(navegador.window_handles[0])
                try:
                    if pedido<10:
                        valor = float(precos_ajustados[pedido])
                        pedido+=1 
                except:
                    print(f'OLHA O ERRO: {precos_ajustados[pedido]}, u: {u}, pedido:{pedido}')
            if pedido>=10:
                #troca de pagina, não mudar
                pedido=0
                navegador.switch_to.window(navegador.window_handles[1])
                passar_pagina_pedidos(u)
                u+=1
                esperar_tempo('class','order-card__total')
                time.sleep(2)
                precos = navegador.find_elements(By.CLASS_NAME,'order-card__total')
                precos_ajustados=lista_precos(precos)
                navegador.switch_to.window(navegador.window_handles[0]) 

        if parceiro == 'Clube Livelo' and valor !=0:
            valor_clube = valor
        if operacao == 'Acúmulo' or operacao=='Bônus':
            operacao= 'Compra'
        else:
            operacao= 'Transferência Saída'

        movimentos[i+o*10]=(data_operacional,operacao,parceiro,pontos,valor)
        atualizacao = True
        # print(movimentos)
        i= i+1
        if i >10:
            #troca de paginas
            passar_pagina_extratos(o)
            o+=1
            i=1

    navegador.get('https://www.livelo.com.br/clube-livelo?occsite=points')
    esperar_tempo('id','wi32900152-user-status-id')
    navegador.switch_to.window(navegador.window_handles[1])
    navegador.switch_to.window(navegador.window_handles[0])
    esperar_tempo('id','wi32900152-user-status-id')
    status_clube= navegador.find_element(By.ID,'wi32900152-user-status-id').text
    while len(status_clube)==0:
        status_clube= navegador.find_element(By.ID,'wi32900152-user-status-id').text
        time.sleep(0.12)
    if status_clube == 'Clube Ativo':
        marcacao =1
    else:
        marcacao=0

    conn =pyodbc.connect(dados_conexao)
    cursor = conn.cursor()

    if len(movimentos)!=0:
        teste={}
        data_save,op_save,parceiro_save,pontos_save,valor_save = movimentos[len(movimentos)]
        for movimento in range(len(movimentos) -1,-1,-1):
            try:
                data , operacao, parceiro, pontos,valor = movimentos[movimento]
            except:
                data= datetime.strptime('01/01/1001', '%d/%m/%Y')
                parceiro= ''
                operacao= ''
            if data_save == data and parceiro_save==parceiro and operacao==op_save:
                if valor == 'Calcular':
                    pontos_save = pontos_save+pontos
                    valor_save = 'Calcular'
                else:
                    valor_save=valor_save+valor
                    pontos_save += pontos
            else:
                if valor_save == 'Calcular':
                    cursor.execute(f"SELECT Id, Transação, Quantidade, Valor FROM Movimentações WHERE Conta = {id_conta} AND Valor IS NOT NULL")
                    dados = cursor.fetchall()
                    colunas = [column[0] for column in cursor.description]
                    movimento_df=pd.DataFrame([tuple(row) for row in dados], columns=colunas)

                    somas = movimento_df.groupby('Transação').sum()
                    c= 'Compra'
                    compra_valor,compra_qtd = verificador_somas(c,somas)
                    te = 'Transferência Entrada'
                    te_valor , te_qtd = verificador_somas(te,somas)
                    ts= 'Transferência Saída'
                    ts_valor , ts_qtd = verificador_somas(ts,somas)
                    v= 'Venda'
                    venda_valor, venda_qtd = verificador_somas(v,somas)
                    if (compra_qtd + te_qtd)-(ts_qtd + venda_qtd) !=0:
                        valor_milheiro = abs(((compra_valor + te_valor)-(ts_valor + venda_valor))/((compra_qtd + te_qtd)-(ts_qtd + venda_qtd)))
                    else:
                        valor_milheiro = 0.022
                    valor_save = pontos_save*valor_milheiro
                #inserir as infos no bd
                valor_save = round(valor_save,2)
                if (op_save == 'Compra') and (parceiro_save in transferiveis):
                    parceiro_save = 'Compra de Pontos'
                query ='''INSERT INTO Movimentações (Conta, Transação, Data_Transação, Data_Financeiro, Quantidade, Valor, Observação) 
                                VALUES (?,?,?,?,?,?,?)'''
                cursor.execute(query,(id_conta,op_save,data_save,data_save,pontos_save,valor_save,parceiro_save))
                conn.commit()
                teste[movimento]=(datetime.strftime(data_save,'%d/%m/%Y'),op_save,parceiro_save,pontos_save,valor_save) 
                try:
                    data_save,op_save,parceiro_save,pontos_save,valor_save = movimentos[movimento]
                except:
                    None
        display(teste)
  
    if valor_clube != 0:
        cursor.execute("UPDATE Contas SET Assinatura = ?, Ultima_Atualização =? WHERE Id = ?, Valor =?",(marcacao,anteontem,id_conta,valor_clube))
    else:
        cursor.execute("UPDATE Contas SET Assinatura = ?, Ultima_Atualização =? WHERE Id = ?",(marcacao,anteontem,id_conta))
    cursor.commit()

    cursor.execute(f"SELECT Id, Transação, Quantidade, Valor FROM Movimentações WHERE Conta = {id_conta} AND Valor IS NOT NULL")
    dados = cursor.fetchall()
    colunas = [column[0] for column in cursor.description]
    movimento_df=pd.DataFrame([tuple(row) for row in dados], columns=colunas)

    somas = movimento_df.groupby('Transação').sum()
    c= 'Compra'
    compra_valor,compra_qtd = verificador_somas(c,somas)
    te = 'Transferência Entrada'
    te_valor , te_qtd = verificador_somas(te,somas)
    ts= 'Transferência Saída'
    ts_valor , ts_qtd = verificador_somas(ts,somas)
    v= 'Venda'
    venda_valor, venda_qtd = verificador_somas(v,somas)
    
    cursor.close()
    conn.close()

    saldo_fisico = compra_qtd + te_qtd - ts_qtd - venda_qtd
    if saldo_fisico == 0:
        saldo_fisico = 1
    financeiro = compra_valor +te_valor - ts_valor - venda_valor

    contas[id_conta]=(atualizacao,status_clube)
    nome = pessoas_df.loc[pessoas_df['Id']==id_pessoa,'Nome'].values[0]
    if atualizacao:
        texto= texto + f'{nome} teve atualização nas movimentações, Status do Clube: {status_clube}\n'

    else:
        texto= texto +f'{nome} não teve atualização nas movimentações, Status do Clube: {status_clube}\n' 

    texto = texto + f'''  Saldo Físico Real: {int(saldo_fisico)}
      Valor Financeiro Acumulado: {round(financeiro,2)}
      Valor do Milheiro: {round(financeiro*1000/saldo_fisico,2)} \n \n'''

    menu= navegador.find_element(By.XPATH,'//*[@id="div-controlLogin"]')
    logout= navegador.find_element(By.ID,'a-logout')
    ActionChains(navegador).move_to_element(menu).perform()
    logout.click()

    time.sleep(4)

    
print(contas)
print(texto)
#LEMBRE DE FAZER A LOGICA DA SOMA DO TEXTO
ontem = datetime.strftime(ontem,'%d/%m/%Y')
outlook=win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'arthurtucapereira@gmail.com'
mail.Subject= f'Registro Livelo - {ontem}'
mail.Body = texto
mail.Send()

outlook=win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'viagens@heix.com.br'
mail.Subject= f'Registro Livelo - {ontem}'
mail.Body = texto
mail.Send()

0
1


{1: ('11/12/2024', 'Compra', 'Compra de Pontos', 990, 27.52),
 0: ('11/12/2024', 'Transferência Saída', 'Azul', 1000, np.float64(27.79))}

2
3
4


{0: ('06/12/2024',
  'Transferência Saída',
  'LATAM Pass',
  90000,
  np.float64(2655.78))}

5


...

6


{2: ('06/12/2024', 'Compra', 'Compra de Pontos', 89100, 2815.55),
 1: ('06/12/2024',
  'Transferência Saída',
  'LATAM Pass',
  90000,
  np.float64(2764.36)),
 0: ('11/12/2024', 'Transferência Saída', 'Azul', 1000, np.float64(30.71))}

7
8
9
10
11


{0: ('11/12/2024', 'Transferência Saída', 'Azul', 1000, np.float64(18.88))}

{88: (True, 'Clube Ativo')}
Helles não teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 194626
      Valor Financeiro Acumulado: 5454.82
      Valor do Milheiro: 28.03 
 
Xavier teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 21
      Valor Financeiro Acumulado: 0.58
      Valor do Milheiro: 27.62 
 
Isabela teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 109761
      Valor Financeiro Acumulado: 3238.9
      Valor do Milheiro: 29.51 
 
Marcos teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 382503
      Valor Financeiro Acumulado: 10785.34
      Valor do Milheiro: 28.2 
 
Hennia teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 39645
      Valor Financeiro Acumulado: 1217.7
      Valor do Milheiro: 30.72 
 
Maicon não teve atualização nas movimentações, Status do Clube: Clube Ativo
  Saldo Físico Real: 

In [37]:
data = navegador.find_element(By.ID,'wi32900152-user-status-id').text
print(data)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wi32900152-user-status-id"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B134A3+25059]
	(No symbol) [0x00A9CEA4]
	(No symbol) [0x0097BEC3]
	(No symbol) [0x009BFD86]
	(No symbol) [0x009BFFCB]
	(No symbol) [0x009FD952]
	(No symbol) [0x009E1F44]
	(No symbol) [0x009FB51E]
	(No symbol) [0x009E1C96]
	(No symbol) [0x009B3FAC]
	(No symbol) [0x009B4F3D]
	GetHandleVerifier [0x00E05613+3113811]
	GetHandleVerifier [0x00E1A2DA+3199002]
	GetHandleVerifier [0x00E12AB2+3168242]
	GetHandleVerifier [0x00BB3310+680016]
	(No symbol) [0x00AA57ED]
	(No symbol) [0x00AA2A98]
	(No symbol) [0x00AA2C35]
	(No symbol) [0x00A95890]
	BaseThreadInitThunk [0x75987BA9+25]
	RtlInitializeExceptionChain [0x779BC0CB+107]
	RtlClearB